In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
hatco = pd.read_csv('data_processing/cleaning_normalizing/hatco_csv.csv', sep=';', decimal=',')

hatco

,id,velocidade_entrega,nivel_preco,flexibilidade_preco,imagem_fornecedor,qualidade_servico,imagem_forca_vendas,qualidade_produto,tamanho_firma,nivel_uso,nivel_satisfacao,area_industria
0,1,4.1,0.6,6.9,4.7,2.4,2.3,5.2,pequena,32.0,4.2,industria
1,2,1.8,3.0,6.3,6.6,2.5,4.0,8.4,grande,43.0,4.3,outra_area
2,3,3.4,5.2,5.7,6.0,4.3,2.7,8.2,grande,48.0,5.2,industria
3,4,2.7,1.0,7.1,5.9,1.8,2.3,7.8,grande,32.0,3.9,industria
4,5,6.0,0.9,9.6,7.8,3.4,4.6,4.5,pequena,58.0,6.8,industria
...,...,...,...,...,...,...,...,...,...,...,...,...
165,266,4.2,2.4,9.4,4.9,3.2,2.7,8.5,pequena,49.0,5.2,industria
166,267,3.8,0.8,NaN,NaN,2.2,2.6,5.3,pequena,42.0,5.1,outra_area
167,268,3.3,2.6,9.7,3.3,2.9,1.5,5.2,pequena,47.0,NaN,industria
168,269,NaN,1.9,NaN,4.5,1.5,3.1,9.9,grande,39.0,3.3,industria


In [6]:
# contar as linhas
missings_por_linha = pd.DataFrame({'id': hatco['id'].tolist(),
                                    'n_missings': hatco.isna().sum(axis=1).tolist()})
n_colunas = hatco.shape[1] - 1

# verificar os resultados
missings_por_linha.assign( perc_missings = missings_por_linha['n_missings'] / n_colunas ) \
    .sort_values('perc_missings', ascending = False) 

,id,n_missings,perc_missings
109,210,7,0.636364
162,263,7,0.636364
160,261,6,0.545455
113,214,6,0.545455
132,233,5,0.454545
...,...,...,...
157,258,0,0.000000
165,266,0,0.000000
164,265,0,0.000000
163,264,0,0.000000


In [7]:
# contar as colunas
missings_por_coluna = hatco.isna().sum()
missings_por_coluna = pd.DataFrame( missings_por_coluna, 
                                    columns=['n']) \
                                    .reset_index() \
                                    .rename(columns={'index': 'variaveis'})
n_linhas = hatco.shape[0]

# verificar os resultados
missings_por_coluna \
    .assign( perc_missings = missings_por_coluna['n'] / n_linhas) \
    .sort_values('n', ascending = False)

,variaveis,n,perc_missings
1,velocidade_entrega,21,0.123529
3,flexibilidade_preco,17,0.100000
2,nivel_preco,13,0.076471
5,qualidade_servico,9,0.052941
9,nivel_uso,9,0.052941
7,qualidade_produto,9,0.052941
10,nivel_satisfacao,7,0.041176
4,imagem_fornecedor,7,0.041176
6,imagem_forca_vendas,6,0.035294
8,tamanho_firma,2,0.011765


In [9]:
# cruzamento das variáveis
pd.crosstab(hatco.tamanho_firma.fillna('NA'), 
            hatco.area_industria.fillna('NA'))

area_industria,NA,industria,outra_area
tamanho_firma,,,
NA,0,0,2
grande,1,32,32
pequena,0,51,52


In [14]:
# correlação eliminando os missings
hatco[['velocidade_entrega', 'qualidade_produto']].corr()

,velocidade_entrega,qualidade_produto
velocidade_entrega,1.000000,-0.404352
qualidade_produto,-0.404352,1.000000


In [15]:
# correlação trocando os missings por zero
hatco[['velocidade_entrega', 'qualidade_produto']].fillna(0).corr()

,velocidade_entrega,qualidade_produto
velocidade_entrega,1.00000,-0.12319
qualidade_produto,-0.12319,1.00000


## Treat missings

In [20]:
hatco.shape, hatco.dropna().shape

((170, 12), (126, 12))

In [21]:
# obter o mais frequente
hatco['area_industria'] \
    .value_counts(dropna=False) \
    .to_frame('n') \
    .reset_index() \
    .rename(columns={'index': 'area_industria'})

,area_industria,n
0,outra_area,86
1,industria,83
2,NaN,1


In [23]:
# imputar dados pelo mais frequente
hatco['area_industria'] \
    .fillna('outra_area') \
    .value_counts(dropna=False) \
    .to_frame('n') \
    .reset_index() \
    .rename(columns={'index': 'area_industria'})

,area_industria,n
0,outra_area,87
1,industria,83


In [26]:
# obter as estatisticas
stats = hatco[['flexibilidade_preco', 'imagem_fornecedor']].describe()

# imputar os dados
hatco2 = hatco \
    .assign(flexibilidade_preco2 = hatco['flexibilidade_preco'].fillna(stats['flexibilidade_preco'].loc['50%']) ) \
    .assign(imagem_fornecedor2 = hatco['imagem_fornecedor'].fillna(stats['imagem_fornecedor'].loc['mean']) )

# ver os resultados
hatco2[['id', 'flexibilidade_preco', 'flexibilidade_preco2', 'imagem_fornecedor', 'imagem_fornecedor2']].tail(5)

,id,flexibilidade_preco,flexibilidade_preco2,imagem_fornecedor,imagem_fornecedor2
165,266,9.4,9.4,4.9,4.900000
166,267,NaN,8.2,NaN,5.217178
167,268,9.7,9.7,3.3,3.300000
168,269,NaN,8.2,4.5,4.500000
169,270,8.7,8.7,4.6,4.600000
